In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, XLNetTokenizer, XLNetForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import seaborn as sns

# Load the preprocessed dataset
df_reviews = pd.read_csv("../data/processed/df_reviews.csv")

In [3]:
df_reviews.shape()

NameError: name 'df_reviews' is not defined

In [47]:
df_reviews.head()

,sentiment,text,tokens
45000,positive,ive worked classic several party say despite l...,"['ive', 'worked', 'classic', 'several', 'party..."
45001,positive,best theater ever digital screen loyalty progr...,"['best', 'theater', 'ever', 'digital', 'screen..."
45002,positive,ive ever came breakfast couple time even thoug...,"['ive', 'ever', 'came', 'breakfast', 'couple',..."
45003,positive,could eat every day go spurt eat several meal ...,"['could', 'eat', 'every', 'day', 'go', 'spurt'..."
45004,positive,staff friendly coffee perfect doughnut always ...,"['staff', 'friendly', 'coffee', 'perfect', 'do..."


In [48]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(df_reviews['text'], df_reviews['sentiment'], test_size=0.2, random_state=42)

In [49]:
# Tokenization for RoBERTa
tokenizer_roberta = RobertaTokenizer.from_pretrained('roberta-base')
train_encodings_roberta = tokenizer_roberta(X_train.tolist(), truncation=True, padding=True, max_length=128, return_tensors="pt")
test_encodings_roberta = tokenizer_roberta(X_test.tolist(), truncation=True, padding=True, max_length=128, return_tensors="pt")

In [50]:
# Tokenization for XLNet
tokenizer_xlnet = XLNetTokenizer.from_pretrained('xlnet-base-cased')
train_encodings_xlnet = tokenizer_xlnet(X_train.tolist(), truncation=True, padding=True, max_length=128, return_tensors="pt")
test_encodings_xlnet = tokenizer_xlnet(X_test.tolist(), truncation=True, padding=True, max_length=128, return_tensors="pt")

In [51]:
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [52]:
# Convert labels to numerical values
label_map = {'negative': 0, 'neutral': 1, 'positive': 2}
y_train_numeric = y_train.map(label_map).tolist()
y_test_numeric = y_test.map(label_map).tolist()

In [53]:
# Create datasets
train_dataset_roberta = CustomDataset(train_encodings_roberta, y_train_numeric)
test_dataset_roberta = CustomDataset(test_encodings_roberta, y_test_numeric)

train_dataset_xlnet = CustomDataset(train_encodings_xlnet, y_train_numeric)
test_dataset_xlnet = CustomDataset(test_encodings_xlnet, y_test_numeric)

In [54]:
# Define the RoBERTa model
model_roberta = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [55]:
# Define the XLNet model
model_xlnet = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=3)


Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [56]:
# Training arguments
training_args = TrainingArguments(
    output_dir='../models/transfer_learning/roberta_model',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='../logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

c:\Users\abdel\anaconda3\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [57]:
# Trainer for RoBERTa
trainer_roberta = Trainer(
    model=model_roberta,
    args=training_args,
    train_dataset=train_dataset_roberta,
    eval_dataset=test_dataset_roberta,
)

In [58]:
# Trainer for XLNet
trainer_xlnet = Trainer(
    model=model_xlnet,
    args=training_args,
    train_dataset=train_dataset_xlnet,
    eval_dataset=test_dataset_xlnet,
)


In [59]:
# Train the models
trainer_roberta.train()

  0%|          | 0/750 [00:00<?, ?it/s]

{'loss': 1.1188, 'grad_norm': 3.486030101776123, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.04}
{'loss': 1.1078, 'grad_norm': 3.9940593242645264, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.08}
{'loss': 1.0737, 'grad_norm': 3.5862300395965576, 'learning_rate': 3e-06, 'epoch': 0.12}
{'loss': 1.0266, 'grad_norm': 4.154654026031494, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.16}
{'loss': 0.8674, 'grad_norm': 3.6694533824920654, 'learning_rate': 5e-06, 'epoch': 0.2}
{'loss': 0.8099, 'grad_norm': 7.234017848968506, 'learning_rate': 6e-06, 'epoch': 0.24}
{'loss': 0.7151, 'grad_norm': 3.921678304672241, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.28}
{'loss': 0.6218, 'grad_norm': 8.32940673828125, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.32}
{'loss': 0.732, 'grad_norm': 12.056319236755371, 'learning_rate': 9e-06, 'epoch': 0.36}
{'loss': 0.6139, 'grad_norm': 20.39013671875, 'learning_rate': 1e-05, 'epoch': 0.4}
{'loss': 0.5167, 'grad_norm': 5.168823242

  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.6459236741065979, 'eval_runtime': 190.199, 'eval_samples_per_second': 5.258, 'eval_steps_per_second': 0.331, 'epoch': 1.0}
{'loss': 0.5459, 'grad_norm': 14.623492240905762, 'learning_rate': 2.6000000000000002e-05, 'epoch': 1.04}
{'loss': 0.4476, 'grad_norm': 6.672553062438965, 'learning_rate': 2.7000000000000002e-05, 'epoch': 1.08}
{'loss': 0.5544, 'grad_norm': 14.787919998168945, 'learning_rate': 2.8000000000000003e-05, 'epoch': 1.12}
{'loss': 0.543, 'grad_norm': 3.346604108810425, 'learning_rate': 2.9e-05, 'epoch': 1.16}
{'loss': 0.4405, 'grad_norm': 3.739861249923706, 'learning_rate': 3e-05, 'epoch': 1.2}
{'loss': 0.5562, 'grad_norm': 29.010997772216797, 'learning_rate': 3.1e-05, 'epoch': 1.24}
{'loss': 0.4257, 'grad_norm': 8.847997665405273, 'learning_rate': 3.2000000000000005e-05, 'epoch': 1.28}
{'loss': 0.3417, 'grad_norm': 11.799880981445312, 'learning_rate': 3.3e-05, 'epoch': 1.32}
{'loss': 0.5539, 'grad_norm': 14.800016403198242, 'learning_rate': 3.400000000000

  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.3971913456916809, 'eval_runtime': 123.0126, 'eval_samples_per_second': 8.129, 'eval_steps_per_second': 0.512, 'epoch': 2.0}
{'loss': 0.3807, 'grad_norm': 2.025066375732422, 'learning_rate': 4.8e-05, 'epoch': 2.04}
{'loss': 0.4677, 'grad_norm': 4.267898082733154, 'learning_rate': 4.600000000000001e-05, 'epoch': 2.08}
{'loss': 0.4534, 'grad_norm': 10.444662094116211, 'learning_rate': 4.4000000000000006e-05, 'epoch': 2.12}
{'loss': 0.4358, 'grad_norm': 12.122603416442871, 'learning_rate': 4.2e-05, 'epoch': 2.16}
{'loss': 0.3377, 'grad_norm': 12.173665046691895, 'learning_rate': 4e-05, 'epoch': 2.2}
{'loss': 0.3194, 'grad_norm': 8.490181922912598, 'learning_rate': 3.8e-05, 'epoch': 2.24}
{'loss': 0.5062, 'grad_norm': 29.916885375976562, 'learning_rate': 3.6e-05, 'epoch': 2.28}
{'loss': 0.271, 'grad_norm': 6.563568592071533, 'learning_rate': 3.4000000000000007e-05, 'epoch': 2.32}
{'loss': 0.3356, 'grad_norm': 16.56073570251465, 'learning_rate': 3.2000000000000005e-05, 'epoch

  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.4276874363422394, 'eval_runtime': 152.0758, 'eval_samples_per_second': 6.576, 'eval_steps_per_second': 0.414, 'epoch': 3.0}
{'train_runtime': 8738.3413, 'train_samples_per_second': 1.373, 'train_steps_per_second': 0.086, 'train_loss': 0.5151433836619059, 'epoch': 3.0}


TrainOutput(global_step=750, training_loss=0.5151433836619059, metrics={'train_runtime': 8738.3413, 'train_samples_per_second': 1.373, 'train_steps_per_second': 0.086, 'total_flos': 789340253184000.0, 'train_loss': 0.5151433836619059, 'epoch': 3.0})

In [ ]:
# Train the models
trainer_xlnet.train()

  0%|          | 0/750 [00:00<?, ?it/s]

{'loss': 1.1046, 'grad_norm': 29.230403900146484, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.04}
{'loss': 1.041, 'grad_norm': 22.902469635009766, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.08}
{'loss': 0.9183, 'grad_norm': 21.140338897705078, 'learning_rate': 3e-06, 'epoch': 0.12}
{'loss': 0.927, 'grad_norm': 20.103660583496094, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.16}
{'loss': 0.7971, 'grad_norm': 25.162105560302734, 'learning_rate': 5e-06, 'epoch': 0.2}
{'loss': 0.8154, 'grad_norm': 25.732370376586914, 'learning_rate': 6e-06, 'epoch': 0.24}
{'loss': 0.7503, 'grad_norm': 23.569629669189453, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.28}
{'loss': 0.6209, 'grad_norm': 22.90118408203125, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.32}
{'loss': 0.8284, 'grad_norm': 11.556195259094238, 'learning_rate': 9e-06, 'epoch': 0.36}
{'loss': 0.7, 'grad_norm': 8.88903522491455, 'learning_rate': 1e-05, 'epoch': 0.4}
{'loss': 0.5854, 'grad_norm': 21.15174

In [ ]:
# Evaluate RoBERTa
predictions_roberta = trainer_roberta.predict(test_dataset_roberta)
y_pred_roberta = np.argmax(predictions_roberta.predictions, axis=1)

  0%|          | 0/63 [00:00<?, ?it/s]

In [ ]:
# Evaluate XLNet
predictions_xlnet = trainer_xlnet.predict(test_dataset_xlnet)
y_pred_xlnet = np.argmax(predictions_xlnet.predictions, axis=1)

  0%|          | 0/63 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# Save the predictions
np.save('../models/transfer_learning/roberta_predictions.npy', y_pred_roberta)
np.save('../models/transfer_learning/xlnet_predictions.npy', y_pred_xlnet)

In [ ]:
# Plot training loss
def plot_training_loss(trainer, model_name):
    plt.figure(figsize=(10, 6))
    plt.plot(trainer.state.log_history, label='Training Loss')
    plt.title(f'{model_name} Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

plot_training_loss(trainer_roberta, 'RoBERTa')
plot_training_loss(trainer_xlnet, 'XLNet')

In [ ]:
# Plot evaluation metrics
def plot_evaluation_metrics(y_true, y_pred, model_name):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')

    metrics = pd.DataFrame({
        'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
        'Score': [accuracy, precision, recall, f1]
    })

    plt.figure(figsize=(10, 6))
    sns.barplot(data=metrics, x='Metric', y='Score')
    plt.title(f'{model_name} Evaluation Metrics')
    plt.xlabel('Metric')
    plt.ylabel('Score')
    plt.show()

plot_evaluation_metrics(y_test_numeric, y_pred_roberta, 'RoBERTa')
plot_evaluation_metrics(y_test_numeric, y_pred_xlnet, 'XLNet')